In [ ]:
def cl_2h_mod(l,chi_i,chi_f,h,omega_m,omega_lambda,omega_b,d_h,N,beta,m_0,z_0,A_s,n_s,k_max):
#     h=0.7
#     omega_m=0.3
#     omega_lambda=0.7
#     d_h=3000/h
#     N=43182 #total number of the galaxies in 2MRS catalog (unmasked)
#     beta=1.64
#     m_0=1.31
#     z_0=0.0266
#              #set the Cosmology

#     Omega_b = 0.05
#     Omega_m = 0.3
#     Omega_cdm = Omega_m - Omega_b
#     h = 0.7
#     A_s = 2.1e-9
#     n_s = 0.96

#     k_max = 200 #UNITS: 1/Mpc

    params = {
                 'output':'mPk',
                 'non linear':'halofit',
                 'Omega_b':Omega_b,
                 'Omega_cdm':Omega_cdm,
                 'h':h,
                 'A_s':A_s,
                 'n_s':n_s,
                 'P_k_max_1/Mpc':k_max,
                 'z_max_pk':10
    }

    #Initialize the cosmology for linear matter power spectra computation
    cosmo = Class()
    cosmo.set(params)
    cosmo.compute()

    def test(chi):
        
        z=z_at_value(cs.comoving_distance,chi[0]*u.Mpc)
        
        dlndz=(N*beta/(z_0*gamma((m_0+1)/beta))*(z/z_0)**m_0*np.exp(-(z/z_0)**beta))/N
        dchidz=d_h/np.sqrt(omega_m*(1+z)**3+omega_lambda)
        w_g=dlndz/dchidz
        k=l/chi


        #return the P_g 2 halo terms in the unit of [Mpc^3]
        lin=cosmo.pk_lin(k, z)

        h=0.7
        sd_log_M=0.15
        M_0=M_min
        cosmology.setCosmology('planck15')
        def b_g_integration(M):
            #get the variables for integration
            dndm=(mass_function.massFunction(M*h, z, mdef = 'vir', model = 'tinker08', q_out = 'dndlnM')*1/(M*h))*(h**4)

            n_cen=1/2*(1+special.erf((np.log10(M)-np.log10(M_min))/sd_log_M))
            n_sat=((M-M_0)/M_1)**alpha*heaviside(M-M_0,1)

            delta_vir=340 #define the cosmology
            m=M*h #set the M_vir=M, convert the unit from solar mass to solar mass * h^-1

            R=(m/(1.16e12*0.3))**(1/3)

        #constants
            R_star=3.135
            m_star=1.07e13


            c=9*(m/m_star)**(-0.13)
            y=R/R_star
            k_hat=(k/h)*R_star*(delta_vir)**(-1/3)
            kappa=k_hat*y/c

            si_c,ci_c=special.sici(kappa*(1+c))
            si_k,ci_k=special.sici(kappa)

        #compute the actual density
            x=np.sin(kappa)*(si_c-si_k)+np.cos(kappa)*(ci_c-ci_k)-np.sin(kappa*c)/(kappa*(1+c))
            f=1/(np.log(1+c)-c/(1+c))
            u=f*x*heaviside(r_max*h-R,1)

            #use the function from the package Colossus to compute the value of linear halo bias of Tinker et al. 2010
            b_1=bias.haloBias(M*h, z,mdef='vir',model='tinker10')
            return dndm*(n_cen+n_sat*u)*b_1

        def n_g_int(M):
            dndm=(mass_function.massFunction(M*h, z, mdef = 'vir', model = 'tinker08', q_out = 'dndlnM')*1/(M*h))*(h**4)
            n_cen=1/2*(1+special.erf((np.log10(M)-np.log10(M_min))/sd_log_M))
            n_sat=((M-M_0)/M_1)**alpha*heaviside(M-M_0,1)
            return dndm*(n_cen+n_sat)

        ng=integrate.quad(n_g_int,M_i,M_f)[0]
        bg=integrate.quad(b_g_integration,M_i,M_f)[0]

        b_g=(1/ng)*bg
        
        h_2=b_g*b_g*lin
        
        return 1/(chi**2)*w_g**2*h_2

#     q = np.linspace(chi_i, chi_f, num=10)
#     y = np.array([test(ki) for ki in q] )
# #     cl_array=integrate.cumtrapz(y, x)
# #     cl= cl_array[-1]-cl_array[0]
#     cl=integrate.trapezoid(y,q)
    cl=integrate.quad(test,chi_i,chi_f)[0]


    return cl

In [ ]:
def u_sat(k,M,z,r_max): # units of the inputs are k=[Mpc^-1], M=[solar mass], r_max=[Mpc]
    h=0.7
    delta_vir=340 #define the cosmology
    m=M*h #set the M_vir=M, convert the unit from solar mass to solar mass * h^-1
    
    R=(m/(1.16e12*0.3))**(1/3)
    
    #constants
    R_star=3.135
    m_star=1.07e13
    
    
    c=9*(m/m_star)**(-0.13)
    y=R/R_star
    k_hat=(k/h)*R_star*(delta_vir)**(-1/3)
    kappa=k_hat*y/c
    
    si_c,ci_c=special.sici(kappa*(1+c))
    si_k,ci_k=special.sici(kappa)

    #compute the actual density
    x=np.sin(kappa)*(si_c-si_k)+np.cos(kappa)*(ci_c-ci_k)-np.sin(kappa*c)/(kappa*(1+c))
    f=1/(np.log(1+c)-c/(1+c))
    
    return f*x*heaviside(r_max*h-R,1)

def hod(M,M_min,M_1,alpha): # units of all Ms are solar mass
    
    #define some constants according to Ando et al.
    sd_log_M=0.15
    M_0=M_min
    
    N_cen=1/2*(1+special.erf((np.log10(M)-np.log10(M_min))/sd_log_M)) # mean number of central galaxies
    
    N_sat=((M-M_0)/M_1)**alpha*heaviside(M-M_0,1) # mean number of satellite galaxies
    
    return N_cen,N_sat

def dn_dm(M,z):
    '''
    The orinigal function mass_function.massFunction receives mass in unit of h^-1*solar mass, and returns the value
    of dn/dlnM, in unit of (Mpc/h)^-3. We can get the value of dn/dM by multiplying dlnM/dM=1/M to dn/dlnM, and
    convert the final value to unit of Mpc^-3
    
    '''
    h=0.7
    cosmology.setCosmology('planck15')
    return (mass_function.massFunction(M*h, z, mdef = 'vir', model = 'tinker08', q_out = 'dndlnM')*1/(M*h))*(h**4)

def n_g(z,M_i,M_f,M_min,M_1,alpha):
    def n_g_int(M):
        cosmology.setCosmology('planck15')
        return dn_dm(M,z)*(hod(M,M_min,M_1,alpha)[0]+hod(M,M_min,M_1,alpha)[1])
    return integrate.quadrature(n_g_int,M_i,M_f)[0]

In [ ]:

def P_g_1h(z,k,M_i,M_f,M_min,M_1,alpha,r_max):
    def int(M):
        dndm=dn_dm(M,z)
        n_sat=hod(M,M_min,M_1,alpha)[1]
        u=u_sat(k,M,z,r_max)
        return dndm*(2*n_sat*u+n_sat**2*(np.abs(u))**2)
    P_g_1=(1/n_g(z,M_i,M_f,M_min,M_1,alpha)**2)*integrate.quad(int,M_i,M_f)[0]
    return P_g_1


def cl_1h_mod(l,chi_i,chi_f):

    #set some constants
    h=0.7
    omega_m=0.308
    omega_lambda=0.69
    omega_k=1-omega_m-omega_lambda
    d_h=3000/h
    N=43182 #total number of the galaxies in 2MRS catalog (unmasked)
    beta=1.64
    m_0=1.31
    z_0=0.0266

    def test(chi):
        z=z_at_value(cs.comoving_distance,chi[0]*u.Mpc)
        dlndz=N*beta/(z_0*gamma((m_0+1)/beta))*(z/z_0)**m_0*np.exp(-(z/z_0)**beta)*1/N
        dchidz=d_h/np.sqrt(omega_m*(1+z)**3+omega_k*(1+z)**2+omega_lambda)
        w_g=dlndz/dchidz
        k=l/chi
        
        return 1/(chi**2)*w_g**2*dchidz*P_g_1h(z,k,M_i,M_f,M_min,M_1,alpha,r_max)

#     x = np.linspace(chi_i, chi_f, num=10)
#     y = np.array([test(ki) for ki in x])
#     cl=integrate.trapezoid(x,y)
    cl=integrate.quad(test,chi_i,chi_f)[0]

    return cl